## **1. Setup**


---



In [4]:
scope = None
target = None

In [ ]:
import chipwhisperer as cw
import time
class CW_Setuper:
    # Param. for chipwhisperer initialization
    hardware_dir = '../../../'                                                              # path to the imported /hardware/victims/firmware folder
    chipwhisperer_init : dict = {
        "SCOPETYPE" : 'OPENADC',                                                            # Scope type ...

        "MAKEDIR" : './makefile_dir_2/',                                                    # Makefile folder relative path
        "CRYPTO_TARGET" : './',                                                             # Crypto target folder
        "FIRMWAREPATH" : f'{hardware_dir}hardware/victims/firmware',                         # Firmware path, path to compiler, simpleserial of chipwhisperer

        "PLATFORM" : 'CWLITEARM',                                                           # Chipwhisperer device type/Platform
        "SS_VER" :'SS_VER_2_1',                                                             # SimpleSerial version, in this case version 2
        "TARGET" :'simpleserial-target',                                                    # Target name, in this case simpleserial-target
    }
    firmware_bash_cmd = f''


    def _init_chipwhisperer_(self):# - Running additional ipynb file, containing basic setup of chipwhisperer, conditional of SETUP_VARS
        SS_VER = self.chipwhisperer_init["SS_VER"]
        PLATFORM = self.chipwhisperer_init["PLATFORM"]
        try:
            if not scope.connectStatus:
                scope.con()
        except NameError:
            scope = cw.scope()
        
        try:
            if SS_VER == "SS_VER_2_1":
                target_type = cw.targets.SimpleSerial2
            elif SS_VER == "SS_VER_2_0":
                raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
            else:
                target_type = cw.targets.SimpleSerial
        except:
            SS_VER="SS_VER_1_1"
            target_type = cw.targets.SimpleSerial
        
        try:
            target = cw.target(scope, target_type)
        except:
            print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
            print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
            scope = cw.scope()
            target = cw.target(scope, target_type)
        print("INFO: Found ChipWhisperer😍")

        if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
            prog = cw.programmers.STM32FProgrammer
        elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
            prog = cw.programmers.XMEGAProgrammer
        elif "neorv32" in PLATFORM.lower():
            prog = cw.programmers.NEORV32Programmer
        elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
            prog = cw.programmers.SAM4SProgrammer
        else:
            prog = None
        time.sleep(0.05)
        scope.default_setup()
        self.scope = scope
        self.target = target
        print("✔️ INFO: Scope and Target, are set😍")

    
    def __init__(self, firmware_reinit = False, re_init_dict = None):
        if re_init_dict is not None:
            for key in re_init_dict.keys():
                self.chipwhisperer_init[key] = re_init_dict[key]
            print("⚠️ Changed, firmware config!!!")
        if firmware_reinit:
            print("⚠️ Reinit, initiated!!!")
            self._init_firmware_()
        self._init_chipwhisperer_()
        print(f"❤️ Target hex: {self.chipwhisperer_init['MAKEDIR']}simpleserial-target-CWLITEARM.hex")
        cw.program_target(self.scope, cw.programmers.STM32FProgrammer, f"{self.chipwhisperer_init['MAKEDIR']}simpleserial-target-CWLITEARM.hex")
        self.reset_target(self.scope)
        print("✔️ Setuper initiated.")
    
    
    def _init_firmware_(self):
        import subprocess
        self.firmware_bash_cmd = '''cd %s | make CRYPTO_TARGET=%s PLATFORM=%s FIRMWAREPATH=%s SS_VER=%s TARGET=%s> output.txt 2>warnings_new.txt'''
        
        # Deactivated block of code, because it throws error, because path
            #print(self.firmware_bash_cmd  % tuple(str(self.chipwhisperer_init[k]) for k in ('MAKEDIR', 'CRYPTO_TARGET', 'PLATFORM', 'FIRMWAREPATH', 'SS_VER',  'TARGET')))
            #result = subprocess.run(self.firmware_bash_cmd
            #                        % tuple(str(self.chipwhisperer_init[k]) for k in ('MAKEDIR', 'CRYPTO_TARGET', 'PLATFORM', 'FIRMWAREPATH', 'SS_VER',  'TARGET')),
            #                        shell=True,
            #                        executable='bin/bash')
        
        subprocess.call(f'firmware_setup.sh %s %s %s %s %s %s'
                        % tuple(str(self.chipwhisperer_init[k]) for k in ('MAKEDIR', 'CRYPTO_TARGET', 'PLATFORM', 'FIRMWAREPATH', 'SS_VER',  'TARGET')),
                        shell=True)
        print(f"✔️ Firmware maked, with command {self.firmware_bash_cmd}.")
    def _scope_disc_(self):
        try:
            self.scope.dis()
            self.target.dis()
            print("✔️ Firmware disconnected.")
        except Exception as e:
            print("❌ Disconection failed because: ",e)


    def reset_target(self, scope):
        PLATFORM = self.chipwhisperer_init["PLATFORM"]
        if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
            self.scope.io.pdic = 'low'
            time.sleep(0.1)
            self.scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
            time.sleep(0.1) #xmega needs more startup time
        elif "neorv32" in PLATFORM.lower():
            raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
        elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
            self.scope.io.nrst = 'low'
            time.sleep(0.25)
            self.scope.io.nrst = 'high_z'
            time.sleep(0.25)
        else:  
            self.scope.io.nrst = 'low'
            time.sleep(0.05)
            self.scope.io.nrst = 'high_z'
            time.sleep(0.05)

In [1]:
print("✔️ The Setuper class succesfuly imported.")

✔️ The Setuper class succesfuly imported.


In [ ]:
#cw_setuper = CW_Setuper(firmware_reinit = True)
#cw_setuper._scope_disc_()
#cw_setuper = None

### Archive

- Defining several importatn variables, alongside relative target's paths
- Transfered to CW_Setuper._init_chipwhisperer_()
<code>
    SCOPETYPE = 'OPENADC'                                                            # Scope type ...
    hadware_dir = '../../../'                                                        # path to the imported /hardware/victims/firmware folder
    MAKEDIR = './makefile_dir_2/'                                                    # Makefile folder relative path
    CRYPTO_TARGET = './'                                                             # Crypto target folder
    FIRMWAREPATH = f'{hadware_dir}hardware/victims/firmware'                         # Firmware path, path to compiler, simpleserial of chipwhisperer
    PLATFORM = 'CWLITEARM'                                                           # Chipwhisperer device type/Platform
    SS_VER='SS_VER_2_1'                                                              # SimpleSerial version, in this case version 2
    TARGET='simpleserial-target'                                                     # Target name, in this case simpleserial-target
</code>

- Running additional ipynb file, containing basic setup of chipwhisperer, conditional of SETUP_VARS
- Transfered to CW_Setuper._init_chipwhisperer_()
<code>
    import chipwhisperer as cw  
    try:  
        if not scope.connectStatus:  
            scope.dis()  
    except:  
        scope = cw.scope()  
    setup_ipynb = '../Setup_Scripts/Setup_Generic.ipynb'   
    %run '../Setup_Scripts/Setup_Generic.ipynb'
</code>

Note to self:
When the scope is created, the device's green LED light starts blinking (on top of the blue LED that probably indicates that the device is connected physically).

We also set the scope to a default setup which means:

- Sets the scope gain to 45dB
- Sets the scope to capture 5000 samples
- Sets the scope offset to 0 (aka it will begin capturing as soon as it is triggered)
- Sets the scope trigger to rising edge
- Outputs a 7.37MHz clock to the target on HS2
- Clocks the scope ADC at 4*7.37MHz. Note that this is synchronous to the target clock on HS2
- Assigns GPIO1 as serial RX
- Assigns GPIO2 as serial TX

- Running an bash program for translation of crypto target program, into format understandable by the device (.hex)
- There are several inputed vars, which are defined in 1.0. Variable Setup
<code>
    %%bash -s "$MAKEDIR" "$CRYPTO_TARGET" "$PLATFORM" "$FIRMWAREPATH" "$SS_VER" "$TARGET"
    cd $1
    make CRYPTO_TARGET=$2 PLATFORM=$3 FIRMWAREPATH=$4 SS_VER=$5 TARGET=$6> output.txt 2>warnings.txt
</code>

- This is an upload of crypto target software unto chipwhisperer, while it is in it's understandable format (.hex)
- Reseting target for wholesome flushing
<code>
    cw.program_target(scope, cw.programmers.STM32FProgrammer, f"{MAKEDIR}simpleserial-target-CWLITEARM.hex")
    reset_target(scope)
</code>